## Magic Number and Elimination Number

Strictly defined, we use 

```
G + 1 - W_A - L_B
```

In [1]:
import os, sys, subprocess, json, time
from pprint import pprint

In [2]:
def fetch_season_data(which_season0, cup):
    cup = cup.lower()
    seas_file = os.path.join('..', '..', '..', 'data', f'gollyx-{cup}-data', f'season{which_season0}', 'season.json')
    if not os.path.exists(seas_file):
        raise Exception(f"Error: season {which_season0} not valid: {seas_file} does not exist")
    with open(seas_file, 'r') as f:
        season0_seas = json.load(f)
    return season0_seas

def fetch_postseason_data(which_season0, cup):
    cup = cup.lower()
    post_file = os.path.join('..', '..', '..', 'data', f'gollyx-{cup}-data', f'season{which_season0}', 'postseason.json')
    if not os.path.exists(post_file):
        raise Exception(f"Error: season {which_season0} not valid: {post_file} does not exist")
    with open(post_file, 'r') as f:
        season0_post = json.load(f)
    return season0_post

def fetch_teams_data(which_season0, cup):
    cup = cup.lower()
    teams_file = os.path.join('..', '..', '..', 'data', f'gollyx-{cup}-data', f'season{which_season0}', 'teams.json')
    if not os.path.exists(teams_file):
        raise Exception(f"Error: season {which_season0} not valid: {teams_file} does not exist")
    with open(teams_file, 'r') as f:
        season0_teams = json.load(f)
    return season0_teams

In [3]:
def get_leagues(team_dat):
    leagues = list(set([t['league'] for t in team_dat]))
    return sorted(leagues)

def get_leagues_divisions(team_dat):
    leagues_divs = list(set([(t['league'], t['division']) for t in team_dat]))
    leagues_divs.sort(key=lambda x: (x[0], x[1]))
    return leagues_divs

def get_league_division_team(team_abbr, team_dat):
    for t in team_dat:
        if t['teamAbbr'] == team_abbr:
            return (t['league'], t['division'])
    return None

def team_name_to_abbr(team_name, team_dat):
    for t in team_dat:
        if t['teamName'].lower()==team_name.lower():
            return t['teamAbbr']
    return None

def team_abbr_to_name(team_abbr, team_dat):
    for t in team_dat:
        if t['teamAbbr'].upper()==team_abbr.upper():
            return t['teamName']
    return None

In [4]:
def get_division_standings(season0, day0, cup):
    """
    Returns a dictionary
    
    keys are tuples:
    (league name, division name)
    
    values are lists of tuples:
    (team abbr, team wins, team losses, team points scored)
    
    {
        ('X League', 'Y Division'):    [ ('EA',  10,  7,  2345),
                                         ('SS',  12,  5,  1234),
                                         ...
                                       ]
    }
    """
    seas_dat = fetch_season_data(season0, cup)
    teams_dat = fetch_teams_data(season0, cup)
    
    team_w = {}
    team_l = {}
    team_s = {}
    
    for team in teams_dat:
        ta = team['teamAbbr']
        team_w[ta] = 0
        team_l[ta] = 0
        team_s[ta] = 0
    
    # Accumulate runs up to and including day0
    for i in range(day0+1):
        today = seas_dat[i]
        for game in today:
            t1a = game['team1Abbr']
            t2a = game['team2Abbr']
            
            t1s = game['team1Score']
            t2s = game['team2Score']
            
            team_s[t1a] += t1s
            team_s[t2a] += t2s
    
    # Accumulate W/L including day0 outcome
    last_day = seas_dat[day0]
    for game in last_day:
        t1a = game['team1Abbr']
        t2a = game['team2Abbr']

        t1s = game['team1Score']
        t2s = game['team2Score']
            
        t1w, t1l = game['team1WinLoss']
        t2w, t2l = game['team2WinLoss']
        if game['team1Score'] > game['team2Score']:
            t1w += 1
            t2l += 1
        else:
            t2w += 1
            t1l += 1
        
        team_w[t1a] = t1w
        team_l[t1a] = t1l
        
        team_w[t2a] = t2w
        team_l[t2a] = t2l
    
    division_standings = {}
    
    lea_div = get_leagues_divisions(teams_dat)
    for ld in lea_div:
        division_standings[ld] = []
    
    for team in teams_dat:
        ta = team['teamAbbr']
        tl = team['league']
        td = team['division']
        k = (tl, td)
        v = (ta, team_w[ta], team_l[ta], team_s[ta])
        division_standings[k].append(v)
    
    for k in division_standings.keys():
        division_standings[k].sort(key = lambda x: (x[1], x[3]), reverse = True)
    
    return division_standings

In [5]:
def print_division_standings(season0, day0, cup, dps):
    standings = get_division_standings(season0, day0, cup)
    for league, division in standings.keys():
        table = standings[(league, division)]
        print(f"{league}, {division}:")
        print("")
        print(f"Team  |  W  |  L  |  G Left")
        print( "------|-----|-----|--------")
        for row in table:
            print(f"{row[0]:5} | {row[1]:>3} | {row[2]:>3} | {dps-day0-1:>3}")
        print("")

In [9]:
######################
# Hellmouth II TODAY
season0 = 2
day0 = 37
cup = 'ii'
dps = 49

In [10]:
print_division_standings(season0, day0, cup, dps)

Cold League, Fire Division:

Team  |  W  |  L  |  G Left
------|-----|-----|--------
SLC   |  22 |  16 |  11
BPT   |  20 |  18 |  11
LBFB  |  20 |  18 |  11
SS    |  16 |  22 |  11
EA    |  14 |  24 |  11

Cold League, Water Division:

Team  |  W  |  L  |  G Left
------|-----|-----|--------
SAC   |  23 |  15 |  11
SDBA  |  21 |  17 |  11
FF    |  20 |  18 |  11
TB    |  18 |  20 |  11
ABQ   |  16 |  22 |  11

Hot League, Fire Division:

Team  |  W  |  L  |  G Left
------|-----|-----|--------
SFBS  |  22 |  16 |  11
MILF  |  20 |  18 |  11
DET   |  19 |  19 |  11
FFF   |  18 |  20 |  11
VV    |  18 |  20 |  11

Hot League, Water Division:

Team  |  W  |  L  |  G Left
------|-----|-----|--------
AA    |  19 |  19 |  11
DECO  |  19 |  19 |  11
BB    |  19 |  19 |  11
OSHA  |  19 |  19 |  11
SGE   |  17 |  21 |  11



## Magic Number, Elimination Number, Wild Card Elimination Number Calculations

Define functions to get magic number for first place team, elimination number for not-first place teams.

Last, define a function that prints a nice table summarizing standings, showing:
* which teams have clinched the division
* which teams would win the wild card
* which teams are in the running for the wild card
* which teams are completely eliminated from the postseason

In [11]:
def print_magic_numbers(season0, day0, cup, dps):
    standings = get_division_standings(season0, day0, cup)
    for k in standings:
        print(f"{k[0]}, {k[1]}:")
        table = standings[k]
        magic = table[0]
        trail = table[1]

        M = dps + 1 - magic[1] - trail[2]
        print(f"Magic Number for {magic[0]} is {M}")
        print("")

In [12]:
def print_elim_numbers(season0, day0, cup, dps):
    standings = get_division_standings(season0, day0, cup)
    for k in standings:
        print(f"{k[0]}, {k[1]}:")
        table = standings[k]
        first = table[0]
        
        for i in range(1,len(table)):
            nth = table[i]

            M = dps + 1 - first[1] - nth[2]
            print(f"Elimination Number for {nth[0]} is {M}")
        print("")

In [13]:
def print_division_standings_w_elim(season0, day0, cup, dps):
    
    print("Standings:")
    print(f"Season {season0+1}, Day {day0+1}")
    print("")
    
    standings = get_division_standings(season0, day0, cup)
    for league, division in standings.keys():
        
        key = (league, division)
        data = standings[key]
        
        print(f"{league}, {division}:")
        print("==================================")
        print("")
        print(f"  Team  |  W  |  L  | Left | Elim # | WC Elim #")
        print( "-----------------------------------------------")
        
        for i, row in enumerate(data):
            prefix = "  "

            elim = ""
            wc_elim = ""
            
            if i==0:
                # First row is first place team (incl. points tiebreaker)
                
                # Determine magic # for first place team
                first_wins = row[1]
                second_losses = data[1][2]
                magic = dps + 1 - first_wins - second_losses
                
                # If the first place team's magic # < 0, they clinched
                if magic < 0 or day0 == dps-1:
                    prefix = "y-"
            
            else:
                # Determine elimination # for not first place teams
                elim = dps + 1 - data[0][1] - row[2]
                
                # Naive approach:
                # Elimination number of 0 or less means you have no hope
                if elim <= 0:
                    prefix = "x-"
                
                # Wild cards:
                # Need to combine and sort non-first-place teams from both divisions
                combined = []
                for lea_, div_ in standings.keys():
                    if lea_==league:
                        for j, item in enumerate(standings[(lea_,div_)]):
                            if j>0:
                                combined.append(item)
                
                combined.sort(key = lambda x: (x[1], x[3]), reverse = True)
                namecol = 0
                if row[namecol]==combined[0][namecol] or row[0]==combined[1][namecol]:
                    prefix = "w-"
                else:
                    # Determine WC elim # for not wc teams
                    wc_elim = dps + 1 - combined[1][1] - row[2]
                    if wc_elim <= 0:
                        prefix = "x-"
                    else:
                        if day0 < dps-1:
                            prefix = "z-"
                        else:
                            prefix = "x-"
                
                elim = str(elim)
                wc_elim = str(wc_elim)
            
            print(f"{prefix}{row[0]:5} | {row[1]:>3} | {row[2]:>3} | {dps-day0-1:>4} | {elim:>6} | {wc_elim:>4}")
            
        print("\n")

In [14]:
######################
# Hellmouth II TODAY
season0 = 2
day0 = 37
cup = 'ii'
dps = 49

In [15]:
#get_division_standings(season0, day0, cup)

Key:

`y-ABC` = clinched the division (1st place)

`w-ABC` = current front-runners to clinch wild card

`z-ABC` = in contention to clinch wild card

`x-ABC` = completely eliminated from winning division or wild card

In [16]:
for day0_ in [day0]:
    print_division_standings_w_elim(season0, day0_, cup, dps)
    print("")

Standings:
Season 3, Day 38

Cold League, Fire Division:

  Team  |  W  |  L  | Left | Elim # | WC Elim #
-----------------------------------------------
  SLC   |  22 |  16 |   11 |        |     
w-BPT   |  20 |  18 |   11 |     10 |     
z-LBFB  |  20 |  18 |   11 |     10 |   12
z-SS    |  16 |  22 |   11 |      6 |    8
z-EA    |  14 |  24 |   11 |      4 |    6


Cold League, Water Division:

  Team  |  W  |  L  | Left | Elim # | WC Elim #
-----------------------------------------------
  SAC   |  23 |  15 |   11 |        |     
w-SDBA  |  21 |  17 |   11 |     10 |     
z-FF    |  20 |  18 |   11 |      9 |   12
z-TB    |  18 |  20 |   11 |      7 |   10
z-ABQ   |  16 |  22 |   11 |      5 |    8


Hot League, Fire Division:

  Team  |  W  |  L  | Left | Elim # | WC Elim #
-----------------------------------------------
  SFBS  |  22 |  16 |   11 |        |     
w-MILF  |  20 |  18 |   11 |     10 |     
z-DET   |  19 |  19 |   11 |      9 |   12
z-FFF   |  18 |  20 |   11 |     